In [1]:

import sys
sys.dont_write_bytecode = True
import pandas as pd
import akshare as ak
import kplot as kp

In [2]:
kchart = kp.kchartDict()


In [3]:
kchart.getData('000002',period='monthly')

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,1991-01-30,000002,14.58,14.51,14.58,14.51,60,1.044150e+05,7.00,1351.00,13.51,0.02
1,1991-02-28,000002,14.59,15.09,15.41,14.59,7491,1.481768e+07,5.65,4.00,0.58,2.67
2,1991-03-30,000002,15.01,13.25,15.01,13.18,3295,5.482310e+06,12.13,-12.19,-1.84,1.17
3,1991-04-30,000002,13.11,11.65,13.11,11.65,992,1.542445e+06,11.02,-12.08,-1.60,0.35
4,1991-05-31,000002,11.59,11.29,11.59,10.99,13918,1.946591e+07,5.15,-3.09,-0.36,4.96
...,...,...,...,...,...,...,...,...,...,...,...,...
400,2024-11-29,000002,9.21,8.60,10.05,8.22,44079402,4.023740e+10,19.72,-7.33,-0.68,45.36
401,2024-12-31,000002,8.53,7.26,8.99,7.25,27858182,2.285698e+10,20.23,-15.58,-1.34,28.67
402,2025-01-27,000002,7.25,7.27,7.70,6.45,35860451,2.535217e+10,17.22,0.14,0.01,36.91
403,2025-02-28,000002,7.05,7.75,8.22,6.88,44371845,3.377603e+10,18.43,6.60,0.48,45.66


In [4]:
kchart.plot_aks('000002','monthly')


In [5]:
kchart['000002']

,日期,股票代码,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率
0,1991-01-30,000002,14.58,14.51,14.58,14.51,60,1.044150e+05,7.00,1351.00,13.51,0.02
1,1991-02-28,000002,14.59,15.09,15.41,14.59,7491,1.481768e+07,5.65,4.00,0.58,2.67
2,1991-03-30,000002,15.01,13.25,15.01,13.18,3295,5.482310e+06,12.13,-12.19,-1.84,1.17
3,1991-04-30,000002,13.11,11.65,13.11,11.65,992,1.542445e+06,11.02,-12.08,-1.60,0.35
4,1991-05-31,000002,11.59,11.29,11.59,10.99,13918,1.946591e+07,5.15,-3.09,-0.36,4.96
...,...,...,...,...,...,...,...,...,...,...,...,...
400,2024-11-29,000002,9.21,8.60,10.05,8.22,44079402,4.023740e+10,19.72,-7.33,-0.68,45.36
401,2024-12-31,000002,8.53,7.26,8.99,7.25,27858182,2.285698e+10,20.23,-15.58,-1.34,28.67
402,2025-01-27,000002,7.25,7.27,7.70,6.45,35860451,2.535217e+10,17.22,0.14,0.01,36.91
403,2025-02-28,000002,7.05,7.75,8.22,6.88,44371845,3.377603e+10,18.43,6.60,0.48,45.66
